# Load packages

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

# Connect to Datalake

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "***ApplicationID***", # Shoul be add or confige using KeyVault
"fs.azure.account.oauth2.client.secret": "***SecretKey***", # Shoul be add or confige using KeyVault
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/***tenant_id***/oauth2/token"} # Shoul be add or confige using KeyVault


dbutils.fs.mount(
source = "abfss://tokio-olympic-data@tokioolympicsdatalab.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokioolymic",
extra_configs = configs)

In [ ]:
%fs
ls "/mnt/tokioolymic"

In [ ]:
spark

# Load and Transform datasets

In [ ]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokioolymic/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokioolymic/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokioolymic/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokioolymic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokioolymic/raw-data/teams.csv")

In [ ]:
athletes.printSchema()

In [ ]:
coaches.printSchema()

In [ ]:
entriesgender.printSchema()

In [ ]:
#entriesgender = entriesgender.withColumn("Female", col("Female").cast(IntegerType())) \
#    .withColumn("Male", col("Male").cast(IntegerType())) \
#    .withColumn("Total", col("Total").cast(IntegerType()))

In [ ]:
entriesgender.printSchema()

In [ ]:
medals.show()

In [ ]:
medals.printSchema()

In [ ]:
teams.show()

In [ ]:
teams.printSchema()

In [ ]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()

In [ ]:
# Calculate the average number of entries by gender for each discipline

average_entries_by_gender = entriesgender.withColumn(
    "Avg_Female", entriesgender["Female"] / entriesgender["Total"]
).withColumn(
    "Avg_Male", entriesgender["Male"] / entriesgender["Total"]
)

average_entries_by_gender.show()

# Send data to tranformed-data

In [ ]:
athletes.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokioolymic/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokioolymic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokioolymic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokioolymic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokioolymic/transformed-data/teams")